In [18]:
import networkx as nx
from datetime import datetime
import time
import scipy as sp
import numpy as np

# Converting input to NetworkX graph

In [31]:
board_color_vals = []
board_group_vals = []

path_to_solved = "inputs/n8_00/n8_00_solved.txt"
path_to_30empty = "inputs/n8_00/n8_00_30empty.txt"
path_to_10empty = "inputs/n8_00/n8_00_10empty.txt"


# input format for nonomino: <color_value>,<group_value>
with open(path_to_30empty, "r") as n_file:
    lines = n_file.readlines()
    
i = 1
empty_count = 0
for line in lines:
    line = line.strip()
    color_value, group_value = line.split(",")
    
    board_color_vals.append(int(color_value))
    board_group_vals.append(int(group_value))
    
    if int(color_value) == 0:
        empty_count += 1
    
    print(str(i) + ": \t" + line + " [color_value:" + color_value + ", group_value: " + group_value + "]")
    i+=1

print("Number of empty cells: ", empty_count)

1: 	9,1 [color_value:9, group_value: 1]
2: 	0,1 [color_value:0, group_value: 1]
3: 	3,1 [color_value:3, group_value: 1]
4: 	0,1 [color_value:0, group_value: 1]
5: 	0,2 [color_value:0, group_value: 2]
6: 	8,3 [color_value:8, group_value: 3]
7: 	4,3 [color_value:4, group_value: 3]
8: 	2,3 [color_value:2, group_value: 3]
9: 	7,3 [color_value:7, group_value: 3]
10: 	1,1 [color_value:1, group_value: 1]
11: 	4,1 [color_value:4, group_value: 1]
12: 	7,1 [color_value:7, group_value: 1]
13: 	8,2 [color_value:8, group_value: 2]
14: 	2,2 [color_value:2, group_value: 2]
15: 	5,2 [color_value:5, group_value: 2]
16: 	6,2 [color_value:6, group_value: 2]
17: 	9,3 [color_value:9, group_value: 3]
18: 	3,3 [color_value:3, group_value: 3]
19: 	3,4 [color_value:3, group_value: 4]
20: 	0,1 [color_value:0, group_value: 1]
21: 	0,1 [color_value:0, group_value: 1]
22: 	0,2 [color_value:0, group_value: 2]
23: 	9,2 [color_value:9, group_value: 2]
24: 	7,2 [color_value:7, group_value: 2]
25: 	5,3 [color_value:5, 

In [32]:
GRID_SIZE = int(len(board_color_vals) ** 0.5)
SQ_GRID = int(GRID_SIZE ** 0.5)
print("GRID_SIZE =",GRID_SIZE, "\nSQ_GRID =", SQ_GRID)

GRID_SIZE = 9 
SQ_GRID = 3


In [33]:
def init_nonomino_board(board_group_vals):
    nonomino = nx.Graph()
    
    # adding nodes
    nonomino.add_nodes_from([
        (i, {"color": 0, "fixed": False}) for i in range(GRID_SIZE ** 2)
    ])

    unfiltered_neigh = []
    # Adding edges 
    # adding traditional row and column edges
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            # 
            row_neighbours = [(i * GRID_SIZE + j, i * GRID_SIZE + x) for x in range(GRID_SIZE) if x != j]
            col_neighbours = [(i * GRID_SIZE + j, x * GRID_SIZE + j) for x in range(GRID_SIZE) if x != i]

            unfiltered_neigh += row_neighbours + col_neighbours

    # adding nonomino specific edges
    nonomino_neighbours = []
    for i in range(0,len(board_group_vals)):
            for j in range(len(board_group_vals)):
                if board_group_vals[i] == board_group_vals[j] and i != j:
                    nonomino_neighbours.append((i, j))

    unfiltered_neigh += nonomino_neighbours

    # delete multiple edges between two nodes
    filtered_neigh = list(set(unfiltered_neigh))
    
    # add the edges to the graph
    nonomino.add_edges_from(filtered_neigh)
    
    return filtered_neigh, nonomino

In [34]:
# convert a given list "board" to networkx board 
def board_to_nx(sudoku, board):
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            idx = i * GRID_SIZE + j
            
            if board[idx] != 0:
                # 'fixed' means that the color of that node cannot be changed (during mutation or crossover)
                sudoku.nodes[idx]['fixed'] = True
                sudoku.nodes[idx]['color'] = board[idx]
    return sudoku

In [35]:
# utility function to display a sudoku board
def print_board(board):
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            print(board.nodes[i * GRID_SIZE + j]['color'], end=' ')
        print()

In [36]:
filtered_neigh, nonomino = init_nonomino_board(board_group_vals)
nonomino = board_to_nx(nonomino, board_color_vals)

In [37]:
print_board(nonomino)

9 0 3 0 0 8 4 2 7 
1 4 7 8 2 5 6 9 3 
3 0 0 0 9 7 5 6 0 
0 0 4 3 0 6 0 8 2 
0 0 0 9 0 0 2 7 4 
0 2 0 5 8 0 9 0 6 
2 0 0 0 4 9 8 1 0 
4 6 0 2 3 1 7 5 0 
8 7 0 0 6 0 3 0 9 


# Getting full adjacency matrix from nx-graph

In [38]:
print("Number of edges: ", nonomino.number_of_edges())
print("Number of nodes: ", nonomino.number_of_nodes())

Number of edges:  822
Number of nodes:  81


In [39]:
sparse_adjacency_matrix = nx.adjacency_matrix(nonomino)

In [40]:
dense_adjacency_matrix = sparse_adjacency_matrix.todense()
adjacency_2d_array = np.squeeze(np.asarray(dense_adjacency_matrix)).tolist()

In [41]:
# Python program for solution of M Coloring 
# problem using backtracking
 
class Graph():
 
    def __init__(self, nodeCount):
        self.V = nodeCount
        self.graph = [[0 for column in range(nodeCount)] for row in range(nodeCount)]
 
    # A utility function to check 
    # if the current color assignment
    # is safe for vertex v
    def isSafe(self, v, colour, c):
        for i in range(self.V):
            if self.graph[v][i] == 1 and colour[i] == c:
                return False
        return True
     
    # A recursive utility function to solve m
    # coloring  problem
    def graphColourUtil(self, m, colour, v):
        if v == self.V:
            return True
 
        for c in range(1, m + 1):
            if self.isSafe(v, colour, c) == True:
                colour[v] = c
                if self.graphColourUtil(m, colour, v + 1) == True:
                    return True
                colour[v] = 0
 
    def graphColouring(self, m):
        colour = [0] * self.V
        if self.graphColourUtil(m, colour, 0) == None:
            return False
 
        # Print the solution
        print("Solution exist and following are the assigned colours:")
        for c in colour:
            print(c, end=" ")
        return True
    
    def getNodeCount(self):
        return self.V

In [42]:
g = Graph(int(GRID_SIZE ** 2))
g.graph = adjacency_2d_array
m = GRID_SIZE

print()

start = datetime.now()
print("Starting at:", start)

g.graphColouring(m)

end = datetime.now()
print("\nFinishing at:", end)

exec_time = end - start

print("exec time %s " % exec_time)


Starting at: 2021-01-26 16:09:49.569612
Solution exist and following are the assigned colours:
1 2 3 4 5 6 7 8 9 5 6 7 1 2 8 9 3 4 3 8 9 6 4 7 1 2 5 2 1 4 3 7 5 6 9 8 7 5 6 2 3 9 8 4 1 8 9 1 5 6 4 3 7 2 4 3 8 7 9 1 2 5 6 9 4 2 8 1 3 5 6 7 6 7 5 9 8 2 4 1 3 
Finishing at: 2021-01-26 16:10:22.848236
exec time 0:00:33.278624 
